In [1]:
import json

with open("/Users/nikhil/policyengine/policyengine-app/src/posts/blog.json") as f:
    blog = json.load(f)



In [37]:
posts = blog[0]["data"]["collection"]["homepagePostsConnection"]["posts"]

from datetime import datetime

FILTERED_PROPERTIES = [
    "title",
    "uniqueSlug",
    "firstPublishedAt",
]

def get_post(post):
    post = {
        "title": post["title"],
        "name": "-".join(post["uniqueSlug"].split("-")[:-1]),
        "date": datetime.fromtimestamp(post["firstPublishedAt"] / 1000).strftime("%Y-%m-%d"),
        "authors": [post["creator"]["name"].lower().replace(" ", "-")],
        "description": post["extendedPreviewContent"]["subtitle"],
        "imageUrl": "https://miro.medium.com/max/1400/" + post["previewImage"]["id"]
    }
    post["filename"] = f"{post['date']}-{post['name']}.md"
    post["image"] = f"{post['name']}.jpg"
    del post["name"]
    del post["imageUrl"]
    return post

filtered_posts = [get_post(post) for post in posts]

In [31]:
IMAGE_FOLDER = "/Users/nikhil/policyengine/policyengine-app/src/images/posts"
# Download the image for each post and put it in the folder under {name}.jpg
from tqdm import tqdm
for post in tqdm(filtered_posts):
    import requests
    import os
    r = requests.get(post["imageUrl"], allow_redirects=True)
    open(os.path.join(IMAGE_FOLDER, post["name"] + ".jpg"), "wb").write(r.content)


100%|██████████| 10/10 [00:05<00:00,  2.00it/s]


In [39]:
with open("output.txt", "w") as f:
    json.dump(filtered_posts, f, indent=4)